In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from generateData import *

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
torch.set_default_tensor_type(torch.DoubleTensor)

## Initialization of DATA

In [4]:
n = 3 #Input dimension
o = 3 #Output dimension

In [5]:
N = 2000
dim = n
Ntrain = 0.9
noisy = False
M = 5
T = 0.1
x_train, y_train, x_test, y_test, h = generate(N,M,T,Ntrain,noisy)

Time step:  0.025


In [6]:
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.from_numpy(x.astype(np.float64))
    self.y = torch.from_numpy(y.astype(np.float64))
    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    return self.x[idx], self.y[idx]
  def __len__(self):
    return self.length #num elementi del dataset

batch_size = 32

trainset = dataset(x_train,y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = dataset(x_test,y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

## Structure preserving approach

In [7]:
import torch.nn.functional as F

class DynamicBlock(nn.Module):
    def __init__(self, nlayer, input, lift):
        super(DynamicBlock, self).__init__()
        
        self.nlayers = nlayer
        self.input = input
        self.lift = lift
        self.DOF = int(self.input * (self.input-1) / 2)

        self.A = nn.ModuleList([nn.Linear(self.input,self.lift,bias=False) for i in range(self.nlayers)])
        self.B = nn.ModuleList([nn.Linear(self.lift,self.DOF,bias=True) for i in range(self.nlayers)])
        self.Biases = nn.ParameterList([nn.Parameter(torch.randn(self.lift)) for i in range(self.nlayers)])
        self.nl = nn.Sigmoid()

        self.dts = nn.Parameter(torch.randn(self.nlayers))

        self.alpha = torch.ones(self.input)

    def buildSkew(self,ff):
        res = torch.zeros((len(ff),self.input,self.input))
        iu1 = torch.triu_indices(self.input,self.input,1)
        res[:,iu1[0],iu1[1]] = ff
        res = res - torch.transpose(res,1,2)
        return res

    def forward(self, x):
        
        for i in range(self.nlayers):
            c = self.A[i](x)+self.Biases[i]
            ff = self.nl(self.B[i](self.nl(self.A[i](x)+self.Biases[i])))
            Mat = self.buildSkew(ff)
            x = x + self.dts[i] * self.alpha @ Mat
        return x

#Linear lifting layer that preserves the sum
class Lift(nn.Module):
    def __init__(self, lift):
        super(Lift, self).__init__()
        self.outputDim = lift
    def forward(self,x):
        I = torch.eye(x.shape[1])
        Z = torch.zeros((x.shape[1],self.outputDim-x.shape[1]))
        liftMat = torch.cat((I,Z),dim=1)
        return x@liftMat

#Projection layer that preserves the sum
class Projection(nn.Module):
    def __init__(self, inputDim, lowerDim):
        super(Projection, self).__init__()
        self.outputDim = lowerDim
        self.input = inputDim
    def forward(self,x):
        I = torch.eye(self.outputDim)
        Z = torch.zeros((self.input-self.outputDim,self.outputDim))
        projMat = torch.cat((I,Z),dim=0)
        y = x @ projMat
        y = y + torch.sum(x[:,self.outputDim:],dim=1).unsqueeze(1)/(self.outputDim)
        #y[:,-1] = y[:,-1] + torch.sum(x[:,self.outputDim:],dim=1)
        return y        

class Network(nn.Module):
    def __init__(self, input, output):
        super(Network, self).__init__()
        
        self.input = input
        self.output = output
        dim = [10]
        lift = max(dim)
        lista = []
        for i in range(len(dim)):
            lista.append(Lift(dim[i]))
            lista.append(DynamicBlock(5,dim[i],20))
            
        lista.append(Projection(dim[-1],o))
        #lista.append(nn.Linear(dim[-1],o))
        self.seq = nn.Sequential(*lista)

    def forward(self, x):
        x = self.seq(x)
        return x

net = Network(n, o)

In [8]:
import torch.optim as optim

criterion = nn.MSELoss()
lr = 0.01
optimizer = torch.optim.Adam(net.parameters(),lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

In [9]:
checkpoint = 5
epochs = 150

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        inputs, outputs = data[0], data[1]
        optimizer.zero_grad()
        loss = 0
        for k in range(M-1):
          inputs = net(inputs)
          loss += criterion(inputs,outputs[:,:,k])

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        if i % checkpoint == 0:  
            print('[%d, %5d] loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / checkpoint))
            running_loss = 0.0

    scheduler.step()
    if epoch%20 == 0 and epoch>0:
      lr=optimizer.param_groups[0]["lr"]
      print(f"\n\nLR: {lr}, EPOCH: {epoch}\n\n")
    
    #scheduler.step()

print('Finished Training')

[1,     1] loss: 3.886694
[1,     6] loss: 4.560088
[1,    11] loss: 2.766092
[1,    16] loss: 1.028732
[1,    21] loss: 0.428134
[1,    26] loss: 0.532532
[1,    31] loss: 0.178654
[1,    36] loss: 0.093805
[1,    41] loss: 0.108096
[1,    46] loss: 0.036501
[1,    51] loss: 0.029010
[1,    56] loss: 0.030139
[2,     1] loss: 0.001798
[2,     6] loss: 0.018016
[2,    11] loss: 0.009961
[2,    16] loss: 0.007945
[2,    21] loss: 0.008231
[2,    26] loss: 0.006627
[2,    31] loss: 0.005275
[2,    36] loss: 0.006280
[2,    41] loss: 0.005537
[2,    46] loss: 0.005520
[2,    51] loss: 0.005710
[2,    56] loss: 0.004890
[3,     1] loss: 0.000727
[3,     6] loss: 0.005013
[3,    11] loss: 0.004462
[3,    16] loss: 0.005374
[3,    21] loss: 0.005194
[3,    26] loss: 0.005294
[3,    31] loss: 0.005536
[3,    36] loss: 0.004711
[3,    41] loss: 0.005755
[3,    46] loss: 0.007208
[3,    51] loss: 0.004906
[3,    56] loss: 0.004199
[4,     1] loss: 0.001175
[4,     6] loss: 0.004559
[4,    11] l

In [10]:
net.eval();

In [11]:
data = next(iter(testloader))
input, output = data[0], data[1]
predSol = torch.zeros((len(input),dim,M-1))
predSol[:,:,0] = net(input)
for i in range(1,M-1):
    predSol[:,:,i] = net(predSol[:,:,i-1])
print("Test error: ",criterion(predSol,output))
print("Sum first output: ",torch.sum(net(input[0:1])))
print("Sum first input: ",torch.sum(input[0:1]))

Test error:  tensor(3.9161e-06, grad_fn=<MseLossBackward0>)
Sum first output:  tensor(1.0000, grad_fn=<SumBackward0>)
Sum first input:  tensor(1.)


# Evaluation

In [12]:
def flowC(y):
    z = torch.from_numpy(y.astype(np.float64)).reshape(1,-1)
    return net(z).detach().numpy().reshape(-1)

y0 = np.random.rand(dim)
y0 = y0/np.sum(y0)
R0 = 1
f = lambda t,y: np.array([-R0*y[0]*y[1],
                            R0*y[0]*y[1] - y[1],
                            y[1]
    ])

point = y0.copy()
tt = 0
it = 10
mat = np.zeros((3,it))
mat[:,0] = point
time = [0]
for c in range(it-1):
    tt+=h
    time.append(tt)
    point = flowC(point)
    mat[:,c+1] = point
sol_true = scipy.integrate.solve_ivp(f,[0,max(time)],y0,method='RK45',t_eval=time,atol=1e-11,rtol=1e-11).y

fig = plt.figure(figsize=(20,10),dpi=600)
plt.plot(time,mat[0,:],'b-o',label="x",markersize=10)
plt.plot(time,mat[1,:],'y-o',label="y",markersize=10)
plt.plot(time,mat[2,:],'g-o',label="z",markersize=10)
plt.plot(time,np.sum(mat,axis=0),'k-o',label="x + y + z",markersize=10)
plt.plot(time,sol_true[0,:],'c--',label="x true",markersize=10)
plt.plot(time,sol_true[1,:],'m--',label="y true",markersize=10)
plt.plot(time,sol_true[2,:],color='lime', linestyle='dashed', label="z true",markersize=10)
plt.plot(time,np.sum(sol_true,axis=0),color="fuchsia",linestyle="dashed",label="x + y + z true",markersize=10)
plt.legend(fontsize=30, loc='upper center', bbox_to_anchor=(0.5, 0.90), ncol=4, fancybox=True, shadow=True)

plt.title("Behaviour SIR model",fontsize=30)
plt.tick_params(axis='both', which='major', labelsize=30)
plt.xlabel("Time",fontsize=30)
plt.ylabel("Value", fontsize=30)

plt.savefig("SIRModel.svg",dpi=600)

plt.show();